<a href="https://colab.research.google.com/github/Beerschtein/Recommended_systems/blob/main/webinar_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Вебинар 6. Двухуровневые модели рекомендаций**

**Как отбирать кандидатов?**


Вариантов множество. Тут нам поможет MainRecommender. Пока в нем реализованы далеко не все возможные способы генерации кандидатов.

Генерируем топ-k кандидатов.
Качество кандидатов измеряем через recall@k.
recall@k показывает какую долю из купленных товаров мы смогли выявить (рекомендовать) нашей моделью.

# Практическая часть

In [1]:
!pip install implicit==0.4.4

     |████████████████████████████████| 1.1 MB 4.2 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3413730 sha256=2f759467cfb0a2a75c904eb09632d6b84d9a1123c1d1c6ec897e28bbd86bb8d3
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


In [2]:
!pip install lightfm

     |████████████████████████████████| 310 kB 5.2 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705396 sha256=4af20354f6fecf3c3d1930763d02e292e965bf402e41a92ed77ffe9ed6c45ea4
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


**Загружаем данные**


In [3]:
!wget 'https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1Qyf3Invdd3jBexEBK2HtIgPtdbdvzFRx' -O full_data.zip

--2022-05-17 18:08:46--  https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1Qyf3Invdd3jBexEBK2HtIgPtdbdvzFRx
Resolving drive.google.com (drive.google.com)... 142.250.159.100, 142.250.159.101, 142.250.159.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.159.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7ig16jd0h1go9lp3u83qgnjnf52ni5m0/1652810925000/06840004559052771215/*/1Qyf3Invdd3jBexEBK2HtIgPtdbdvzFRx?e=download [following]
--2022-05-17 18:08:46--  https://doc-00-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7ig16jd0h1go9lp3u83qgnjnf52ni5m0/1652810925000/06840004559052771215/*/1Qyf3Invdd3jBexEBK2HtIgPtdbdvzFRx?e=download
Resolving doc-00-18-docs.googleusercontent.com (doc-00-18-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connecting to doc-00-18-docs.googleuserc

In [4]:
!unzip full_data.zip

Archive:  full_data.zip
  inflating: recommenders.py         
  inflating: retail_train.csv        
  inflating: transaction_data.csv    
  inflating: utils.py                
  inflating: hh_demographic.csv      
  inflating: metrics.py              
  inflating: product.csv             


**Import libs**

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

**Read data**

In [6]:
data = pd.read_csv('./retail_train.csv')
item_features = pd.read_csv('./product.csv')
user_features = pd.read_csv('./hh_demographic.csv')

**Process features dataset**

In [7]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'

In [8]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

**Split dataset for train, eval, test**

In [9]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)


VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [10]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [11]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [12]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2108779, 12) Users: 2498 Items: 83685
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [13]:
# выше видим разброс по пользователям и товарам

In [14]:
data_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


**Prefilter items**

In [15]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/content/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 83685 to 5001


**Make cold-start to warm-start**

In [16]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (861404, 13) Users: 2495 Items: 5001
val_matcher
Shape: (169615, 12) Users: 2151 Items: 27644
train_ranker
Shape: (169615, 12) Users: 2151 Items: 27644
val_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


In [17]:
# Теперь warm-start по пользователям

**Init/train recommender**

In [18]:
recommender = MainRecommender(data_train_matcher)

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

**Варианты, как получить кандидатов**

*Можно потом все эти варианты соединить в один*

*(!) Если модель рекомендует < N товаров, то рекомендации дополняются топ-популярными товарами до N*

In [19]:
# Берем тестового юзера 2375

In [20]:
recommender.get_als_recommendations(2375, N=5)

[933354, 834158, 1128333, 9677366, 872530]

In [21]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [22]:
recommender.get_similar_items_recommendation(2375, N=5)

[1120559, 1048918, 1092502, 874651, 1029964]

In [23]:
# Этот код закомментирован потому что долгосчитается!

# recommender.get_similar_users_recommendation(2375, N=5)

**Eval recall of matching**

**Измеряем recall@k**

*Это будет в ДЗ:*

*A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?*

Пока пробуем отобрать 50 кандидатов (k=50).
Качество измеряем на data_val_matcher: следующие 6 недель после трейна

*Дают ли own recommendtions + top-popular лучший recall?*

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500} *Курсив*

C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным? *Курсив*

In [24]:
ACTUAL_COL = 'actual'

In [25]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [26]:
# N = Neighbors
N_PREDICT = 50 

In [27]:
%%time
# для понятности расписано все в строчку, без функций, ваша задача уметь оборачивать все это в функции
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
result_eval_matcher['sim_item_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=50))
result_eval_matcher['als_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=50))

CPU times: user 49.3 s, sys: 27.7 s, total: 1min 17s
Wall time: 47.5 s


In [68]:
%%time
result_eval_matcher['sim_user_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_users_recommendation(x, N=50))

CPU times: user 11min 26s, sys: 3min 43s, total: 15min 10s
Wall time: 11min 7s


In [69]:
result_eval_matcher

,user_id,actual,own_rec,sim_item_rec,als_rec,sim_user_rec
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[9296957, 1032512, 854754, 920091, 966684, 878...","[9244977, 9677366, 1007752, 9677690, 933354, 9...","[958023, 7152455, 7167836, 901067, 5591777, 92..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[981288, 919534, 1048918, 963354, 12487154, 91...","[933354, 9677366, 845914, 9244977, 891531, 801...","[1125601, 9837076, 998741, 1054402, 908314, 97..."
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67...","[6391541, 1052294, 891423, 936470, 1137010, 11...","[5575888, 831557, 823576, 997415, 978332, 8408...","[9677366, 1004945, 834158, 982449, 847139, 841...","[961730, 918046, 918046, 820923, 864335, 57078..."
3,6,"[1024306, 1102949, 6548453, 835394, 940804, 96...","[13003092, 972416, 995598, 923600, 1138596, 10...","[12385373, 919534, 12263692, 1133745, 884686, ...","[7025046, 933354, 1131382, 872530, 9446216, 90...","[896666, 12946027, 9338009, 829563, 1092937, 7..."
4,7,"[836281, 843306, 845294, 914190, 920456, 93886...","[998519, 894360, 7147142, 9338009, 896666, 939...","[5575888, 1048918, 861706, 993466, 966778, 113...","[834158, 1090507, 967461, 951727, 5565392, 104...","[1086740, 6552318, 1019637, 949345, 6632283, 7..."
...,...,...,...,...,...,...
2146,2496,"[831509, 867188, 1013623, 1048851, 5592734, 16...","[872826, 983665, 991546, 1134296, 7441210, 974...","[963354, 919534, 6904419, 9527290, 1048918, 82...","[933354, 9677366, 834158, 9677690, 1131382, 91...","[850676, 953539, 1044487, 831246, 848684, 8531..."
2147,2497,"[820291, 824759, 838797, 859010, 859075, 86077...","[870515, 1117219, 1102207, 1057168, 1135834, 1...","[954994, 1114251, 1056778, 9245108, 1053329, 1...","[933354, 1029685, 1045914, 1065017, 1039927, 9...","[828588, 1114653, 963686, 1048237, 5590965, 11..."
2148,2498,"[865511, 962991, 1076374, 1102358, 5564901, 15...","[1022066, 1076580, 1100379, 5565356, 931579, 8...","[5567037, 1061982, 1062128, 911401, 1139638, 8...","[1070559, 9803591, 7166862, 8019915, 13008459,...","[1012701, 972191, 1112242, 1121808, 1058754, 8..."
2149,2499,"[861282, 921744, 1050968, 13842089, 828837, 86...","[7168055, 1128395, 6904613, 5570048, 889989, 8...","[1061982, 1013703, 905087, 1000365, 827999, 11...","[5565392, 872530, 1007752, 834158, 1102003, 10...","[1071196, 7169088, 1095964, 894360, 6533681, 9..."


In [70]:
result_eval_matcher[['own_rec', 'sim_item_rec']].iloc[0]

own_rec         [856942, 9297615, 5577022, 877391, 9655212, 88...
sim_item_rec    [9296957, 1032512, 854754, 920091, 966684, 878...
Name: 0, dtype: object

In [71]:

def rec_list_concat(*lists, res_len=N_PREDICT):
    res = []
    for i in range(len(lists[0])):
        for j in range(len(lists)):
            res.append(lists[j][i])
    return res[:res_len]


result_eval_matcher['als_and_own_rec'] = list(map(rec_list_concat, result_eval_matcher['als_rec'], result_eval_matcher['own_rec']))
result_eval_matcher['own_and_als_rec'] = list(map(rec_list_concat, result_eval_matcher['own_rec'], result_eval_matcher['als_rec']))
result_eval_matcher['own_and_sim_item_rec'] = list(map(rec_list_concat, result_eval_matcher['own_rec'],result_eval_matcher['sim_item_rec']))
result_eval_matcher['als_and_sim_item_rec'] = list(map(rec_list_concat, result_eval_matcher['own_rec'], result_eval_matcher['als_rec']))
result_eval_matcher

,user_id,actual,own_rec,sim_item_rec,als_rec,sim_user_rec,als_and_own_rec,own_and_als_rec,own_and_sim_item_rec,als_and_sim_item_rec
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[9296957, 1032512, 854754, 920091, 966684, 878...","[9244977, 9677366, 1007752, 9677690, 933354, 9...","[958023, 7152455, 7167836, 901067, 5591777, 92...","[9244977, 856942, 9677366, 9297615, 1007752, 5...","[856942, 9244977, 9297615, 9677366, 5577022, 1...","[856942, 9296957, 9297615, 1032512, 5577022, 8...","[856942, 9244977, 9297615, 9677366, 5577022, 1..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[981288, 919534, 1048918, 963354, 12487154, 91...","[933354, 9677366, 845914, 9244977, 891531, 801...","[1125601, 9837076, 998741, 1054402, 908314, 97...","[933354, 911974, 9677366, 1076580, 845914, 110...","[911974, 933354, 1076580, 9677366, 1103898, 84...","[911974, 981288, 1076580, 919534, 1103898, 104...","[911974, 933354, 1076580, 9677366, 1103898, 84..."
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67...","[6391541, 1052294, 891423, 936470, 1137010, 11...","[5575888, 831557, 823576, 997415, 978332, 8408...","[9677366, 1004945, 834158, 982449, 847139, 841...","[961730, 918046, 918046, 820923, 864335, 57078...","[9677366, 6391541, 1004945, 1052294, 834158, 8...","[6391541, 9677366, 1052294, 1004945, 891423, 8...","[6391541, 5575888, 1052294, 831557, 891423, 82...","[6391541, 9677366, 1052294, 1004945, 891423, 8..."
3,6,"[1024306, 1102949, 6548453, 835394, 940804, 96...","[13003092, 972416, 995598, 923600, 1138596, 10...","[12385373, 919534, 12263692, 1133745, 884686, ...","[7025046, 933354, 1131382, 872530, 9446216, 90...","[896666, 12946027, 9338009, 829563, 1092937, 7...","[7025046, 13003092, 933354, 972416, 1131382, 9...","[13003092, 7025046, 972416, 933354, 995598, 11...","[13003092, 12385373, 972416, 919534, 995598, 1...","[13003092, 7025046, 972416, 933354, 995598, 11..."
4,7,"[836281, 843306, 845294, 914190, 920456, 93886...","[998519, 894360, 7147142, 9338009, 896666, 939...","[5575888, 1048918, 861706, 993466, 966778, 113...","[834158, 1090507, 967461, 951727, 5565392, 104...","[1086740, 6552318, 1019637, 949345, 6632283, 7...","[834158, 998519, 1090507, 894360, 967461, 7147...","[998519, 834158, 894360, 1090507, 7147142, 967...","[998519, 5575888, 894360, 1048918, 7147142, 86...","[998519, 834158, 894360, 1090507, 7147142, 967..."
...,...,...,...,...,...,...,...,...,...,...
2146,2496,"[831509, 867188, 1013623, 1048851, 5592734, 16...","[872826, 983665, 991546, 1134296, 7441210, 974...","[963354, 919534, 6904419, 9527290, 1048918, 82...","[933354, 9677366, 834158, 9677690, 1131382, 91...","[850676, 953539, 1044487, 831246, 848684, 8531...","[933354, 872826, 9677366, 983665, 834158, 9915...","[872826, 933354, 983665, 9677366, 991546, 8341...","[872826, 963354, 983665, 919534, 991546, 69044...","[872826, 933354, 983665, 9677366, 991546, 8341..."
2147,2497,"[820291, 824759, 838797, 859010, 859075, 86077...","[870515, 1117219, 1102207, 1057168, 1135834, 1...","[954994, 1114251, 1056778, 9245108, 1053329, 1...","[933354, 1029685, 1045914, 1065017, 1039927, 9...","[828588, 1114653, 963686, 1048237, 5590965, 11...","[933354, 870515, 1029685, 1117219, 1045914, 11...","[870515, 933354, 1117219, 1029685, 1102207, 10...","[870515, 954994, 1117219, 1114251, 1102207, 10...","[870515, 933354, 1117219, 1029685, 1102207, 10..."
2148,2498,"[865511, 962991, 1076374, 1102358, 5564901, 15...","[1022066, 1076580, 1100379, 5565356, 931579, 8...","[5567037, 1061982, 1062128, 911401, 1139638, 8...","[1070559, 9803591, 7166862, 8019915, 13008459,...","[1012701, 972191, 1112242, 1121808, 1058754, 8...","[1070559, 1022066, 9803591, 1076580, 7166862, ...","[1022066, 1070559, 1076580, 9803591, 1100379, ...","[1022066, 5567037, 1076580, 1061982, 1100379, ...","[1022066, 1070559, 1076580, 9803591, 1100379, ..."
2149,2499,"[861282, 921744, 10

**Пример оборачивания**

In [29]:
# # сырой и простой пример как можно обернуть в функцию
def evalRecall(df_result, target_col_name, recommend_model):
    result_col_name = 'result'
    df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=25))
    return df_result.apply(lambda row: recall_at_k(row[result_col_name], row[ACTUAL_COL], k=N_PREDICT), axis=1).mean()

In [30]:
# evalRecall(result_eval_matcher, USER_COL, recommender.get_own_recommendations)

In [31]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [32]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

**Recall@50 of matching**

In [72]:
TOPK_RECALL = 50

In [73]:
sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)

[('own_rec', 0.06525657038145175),
 ('als_and_own_rec', 0.04557934934513086),
 ('own_and_als_rec', 0.04557934934513086),
 ('als_and_sim_item_rec', 0.04557934934513086),
 ('own_and_sim_item_rec', 0.045523472596337766),
 ('sim_item_rec', 0.011098121582588898),
 ('als_rec', 0.003788036598380074),
 ('sim_user_rec', 0.0017536355770360439)]

**Посчитаем RECALL для количества кандидатов: 20, 50, 100, 200 и 5007**

In [74]:
{20: sorted(calc_recall(result_eval_matcher, 20), key=lambda x: x[1],reverse=True),
 50: sorted(calc_recall(result_eval_matcher, 50), key=lambda x: x[1],reverse=True),
 100: sorted(calc_recall(result_eval_matcher, 100), key=lambda x: x[1],reverse=True),
 200: sorted(calc_recall(result_eval_matcher, 200), key=lambda x: x[1],reverse=True),
 500: sorted(calc_recall(result_eval_matcher, 500), key=lambda x: x[1],reverse=True),
 }

{20: [('own_rec', 0.03928427679372909),
  ('own_and_sim_item_rec', 0.02740491826117513),
  ('als_and_own_rec', 0.02739888994527717),
  ('own_and_als_rec', 0.02739888994527717),
  ('als_and_sim_item_rec', 0.02739888994527717),
  ('sim_item_rec', 0.003579182085286308),
  ('als_rec', 0.001179252224999948),
  ('sim_user_rec', 0.000745907842459949)],
 50: [('own_rec', 0.06525657038145175),
  ('als_and_own_rec', 0.04557934934513086),
  ('own_and_als_rec', 0.04557934934513086),
  ('als_and_sim_item_rec', 0.04557934934513086),
  ('own_and_sim_item_rec', 0.045523472596337766),
  ('sim_item_rec', 0.011098121582588898),
  ('als_rec', 0.003788036598380074),
  ('sim_user_rec', 0.0017536355770360439)],
 100: [('own_rec', 0.06525657038145175),
  ('als_and_own_rec', 0.04557934934513086),
  ('own_and_als_rec', 0.04557934934513086),
  ('als_and_sim_item_rec', 0.04557934934513086),
  ('own_and_sim_item_rec', 0.045523472596337766),
  ('sim_item_rec', 0.011098121582588898),
  ('als_rec', 0.0037880365983800

*Видим, что recall_at_k растет вместе с возрастанием значения k, что логично, так как расширяем число рекомендаций (если будем рекомендовать все товары, recall формально будет 100%). На значении 200 темп роста замедляется. Поэтому предполагаю, что 200 является оптимальным числом кандидиатов.*

**Precision@5 of matching¶**

In [75]:
TOPK_PRECISION = 5

In [76]:
sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

[('own_rec', 0.17712691771268974),
 ('own_and_sim_item_rec', 0.12784751278474998),
 ('own_and_als_rec', 0.12691771269177),
 ('als_and_sim_item_rec', 0.12691771269177),
 ('als_and_own_rec', 0.09502556950255635),
 ('sim_item_rec', 0.0050209205020920475),
 ('als_rec', 0.003626220362622038),
 ('sim_user_rec', 0.002603440260344027)]

**Ranking part¶**

**Обучаем модель 2-ого уровня на выбранных кандидатах**

*Обучаем на data_train_ranking*

*Обучаем только на выбранных кандидатах*

*Я для примера сгенерирую топ-50 кадидиатов через get_own_recommendations*

*(!) Если юзер купил < 50 товаров, то get_own_recommendations дополнит рекоммендации топ-популярными*

In [37]:
# 3 временных интервала
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 

**Подготовка данных для трейна**

In [77]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [78]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [80]:
df_match_candidates.head(2)

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263, 944..."
1,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."


In [82]:
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [83]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [84]:
df_match_candidates.head(4)

,user_id,item_id
0,2070,1105426
0,2070,1097350
0,2070,879194
0,2070,948640


**Check warm start**

In [85]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (107550, 2) Users: 2151 Items: 4574


**Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1**

In [86]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

In [87]:
df_ranker_train.head()

,user_id,item_id,target
2104867,2070,1019940,1
2107468,2021,840361,1
2107469,2021,856060,1
2107470,2021,869344,1
2107471,2021,896862,1


*Не хватает нулей в датасете, поэтому добавляем наших кандитатов в качество нулей*

In [88]:
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

In [89]:
df_ranker_train.target.value_counts()

0.0    99177
1.0     7795
Name: target, dtype: int64

In [90]:
df_ranker_train.head(2)

,user_id,item_id,target
0,2070,1105426,0.0
1,2070,1097350,0.0


*(!) На каждого юзера 50 item_id-кандидатов*

In [91]:
df_ranker_train['target'].mean()

0.07286953595333358

**Подготавливаем фичи для обучения модели**

In [92]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [93]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [94]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [95]:
data

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0


*Фичи user_id: - Средний чек - Средняя сумма покупки 1 товара в каждой категории - Кол-во покупок в каждой категории - Частотность покупок раз/месяц - Долю покупок в выходные - Долю покупок утром/днем/вечером*

In [97]:
#Средний чек
sales_value = data[['user_id', 'sales_value']]
sales_value = sales_value.groupby('user_id')['sales_value'].sum().reset_index()
df_ranker_train = df_ranker_train.merge(sales_value, on='user_id', how='left')
df_ranker_train

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,sales_value
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86
2,2070,879194,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86
3,2070,948640,0.0,1213,DRUG GM,National,ORAL HYGIENE PRODUCTS,WHITENING SYSTEMS,3 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86
4,2070,928263,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,13 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106967,1745,903454,0.0,1216,MEAT-PCKGD,National,FROZEN MEAT,OTHER - FULLY COOKED,32 OZ,45-54,A,Under 15K,Unknown,Single Male,2,None/Unknown,2417.12
106968,1745,9419888,0.0,759,GROCERY,National,YOGURT,YOGURT MULTI-PACKS,48 OZ,45-54,A,Under 15K,Unknown,Single Male,2,None/Unknown,2417.12
106969,1745,1076769,0.0,3859,DELI,National,DELI MEATS,MEAT: LUNCHMEAT BULK,,45-54,A,Under 15K,Unknown,Single Male,2,None/Unknown,2417.12
106970,1745,1092588,0.0,709,GROCERY,National,FLUID MILK PRODUCTS,MISCELLANEOUS MILK,32 OZ,45-54,A,Under 15K,Unknown,Single Male,2,None/Unknown,2417.12


In [98]:

#Средние покупки в месяц по пользователю (частота покупок в месяц)
by_per_week_on_user = data[['user_id', 'week_no', 'item_id']]
by_per_week_on_user = by_per_week_on_user.groupby(['user_id', 'week_no'])['item_id'].nunique().reset_index()
by_per_week_on_user.rename(columns={'item_id': 'buy_count_per_week_on_user'}, inplace=True)
by_per_week_on_user['month_no'] = (by_per_week_on_user['week_no']-1)//4+1
by_per_month_on_user = by_per_week_on_user.groupby(['user_id', 'month_no'])['buy_count_per_week_on_user'].sum().reset_index()
by_per_month_on_user.rename(columns={'buy_count_per_week_on_user': 'buy_per_month_on_user'}, inplace=True)
max_month = by_per_month_on_user['month_no'].max()
by_per_month_on_user = by_per_month_on_user.groupby('user_id')['buy_per_month_on_user'].sum().reset_index()
by_per_month_on_user['buy_per_month_on_user'] = by_per_month_on_user['buy_per_month_on_user']/max_month
by_per_month_on_user.rename(columns={'buy_per_month_on_user': 'average_month_buy_on_user'}, inplace=True)
by_per_month_on_user

,user_id,average_month_buy_on_user
0,1,66.166667
1,2,27.208333
2,3,36.750000
3,4,12.416667
4,5,9.000000
...,...,...
2494,2496,59.125000
2495,2497,79.375000
2496,2498,34.250000
2497,2499,40.291667


*Фичи item_id: - Кол-во покупок в неделю - Среднее кол-во покупок 1 товара в категории в неделю - (Кол-во покупок в неделю) / (Среднее ол-во покупок 1 товара в категории в неделю) - Цена (Можно посчитать из retil_train.csv) - Цена / Средняя цена товара в категории*

*Фичи пары user_id - item_id - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id) - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю) - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)*

In [99]:
df_ranker_train = df_ranker_train.merge(by_per_month_on_user, on='user_id', how='left')
df_ranker_train.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,sales_value,average_month_buy_on_user
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667
2,2070,879194,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667
3,2070,948640,0.0,1213,DRUG GM,National,ORAL HYGIENE PRODUCTS,WHITENING SYSTEMS,3 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667
4,2070,928263,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,13 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667


In [101]:

#Цена
data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
price = data.groupby('item_id')['price'].mean().reset_index()
df_ranker_train = df_ranker_train.merge(price, on='item_id', how='left')
list(df_ranker_train['department'].unique())

['DELI',
 'GROCERY',
 'DRUG GM',
 'MEAT-PCKGD',
 'SEAFOOD-PCKGD',
 'NUTRITION',
 'PRODUCE',
 'PASTRY',
 'MEAT',
 'SEAFOOD',
 'SPIRITS',
 'FLORAL',
 'COSMETICS',
 'MISC. TRANS.']

In [102]:
data = data.merge(item_features[['item_id', 'department']], on='item_id', how='left')
data

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,department
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,1.390000,PRODUCE
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,0.820000,PRODUCE
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,0.990000,PRODUCE
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,1.210000,PRODUCE
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,1.500000,PRODUCE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0,2.000000,GROCERY
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0,1.690000,GROCERY
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0,1.690000,GROCERY
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0,0.001959,KIOSK-GAS


In [103]:
#Промежуточный датафрейм для вычисления средних покупок по категории по конкретным юзерам
buy_cat_reduce_average = data.groupby(['user_id', 'week_no', 'department'])['item_id'].count().reset_index()
buy_cat_reduce_average.rename(columns={'item_id': 'count_per_cat_on_user'}, inplace=True)
buy_cat_reduce_average = buy_cat_reduce_average.groupby(['user_id', 'department'])['count_per_cat_on_user'].mean().reset_index()

In [104]:
#Промежуточный датафрейм для вычисления средних покупок по категории по всем юзерам
buy_cat_reduce_average_all_users = data.groupby(['department', 'week_no'])['item_id'].count().reset_index()
buy_cat_reduce_average_all_users.rename(columns={'item_id': 'count_per_cat'}, inplace=True)
buy_cat_reduce_average_all_users = buy_cat_reduce_average_all_users.groupby('department')['count_per_cat'].mean().reset_index()
buy_cat_reduce_average_all_users

,department,count_per_cat
0,,75.894737
1,AUTOMOTIVE,1.404762
2,CHARITABLE CONT,2.000000
3,CHEF SHOPPE,9.865672
4,CNTRL/STORE SUP,1.750000
5,COSMETICS,74.557895
6,COUP/STR & MFG,8.380435
7,DAIRY DELI,10.600000
8,DELI,611.484211
9,DELI/SNACK BAR,1.125000


In [105]:
buy_cat_reduce_average = buy_cat_reduce_average.merge(buy_cat_reduce_average_all_users, on='department')
buy_cat_reduce_average['average_subtraction_on_cat'] = buy_cat_reduce_average['count_per_cat_on_user'] - buy_cat_reduce_average['count_per_cat']
buy_cat_reduce_average['average_division_on_cat'] = buy_cat_reduce_average['count_per_cat_on_user']/buy_cat_reduce_average['count_per_cat']
buy_cat_reduce_average
#Делая последние 2 фичи из списка, мы получаем еще 4 фичи, которые и замерджим в df_ranker_train

,user_id,department,count_per_cat_on_user,count_per_cat,average_subtraction_on_cat,average_division_on_cat
0,1,,1.090909,75.894737,-74.803828,0.014374
1,6,,1.000000,75.894737,-74.894737,0.013176
2,7,,1.000000,75.894737,-74.894737,0.013176
3,8,,1.142857,75.894737,-74.751880,0.015058
4,13,,1.400000,75.894737,-74.494737,0.018447
...,...,...,...,...,...,...
31757,2318,HBC,1.000000,1.000000,0.000000,1.000000
31758,1613,ELECT &PLUMBING,1.000000,1.000000,0.000000,1.000000
31759,1747,PORK,1.000000,1.000000,0.000000,1.000000
31760,2128,PORK,1.000000,1.000000,0.000000,1.000000


In [106]:
# Надо проверить чтобы все фичи замерджились и добавились
df_ranker_train = df_ranker_train.merge(buy_cat_reduce_average, on=['user_id', 'department'])
df_ranker_train

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,household_size_desc,kid_category_desc,sales_value,average_month_buy_on_user,average_item_buy_on_week,price,count_per_cat_on_user,count_per_cat,average_subtraction_on_cat,average_division_on_cat
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.745763,3.796408,1.333333,611.484211,-610.150877,0.002180
1,2070,1016298,0.0,4074,DELI,National,CHICKEN/POULTRY,CHIX:VALUE ADDED (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.563636,3.101043,1.333333,611.484211,-610.150877,0.002180
2,2070,8015407,0.0,1216,DELI,National,SANDWICHES,SANDWICHES - (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.971831,2.857000,1.333333,611.484211,-610.150877,0.002180
3,2070,1054185,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.631579,3.797258,1.333333,611.484211,-610.150877,0.002180
4,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,...,1,None/Unknown,6981.86,90.416667,1.275000,11.460588,18.939024,15989.789474,-15970.850449,0.001184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105548,1745,1135566,0.0,69,PASTRY,Private,PIES,PIES: FRUIT/NUT,,45-54,...,2,None/Unknown,2417.12,35.916667,2.600000,5.172527,1.200000,371.326316,-370.126316,0.003232
105549,1745,8011291,0.0,69,PASTRY,Private,PIES,PIES: FRUIT/NUT,,45-54,...,2,None/Unknown,2417.12,35.916667,2.609756,5.254486,1.200000,371.326316,-370.126316,0.003232
105550,1745,1107039,0.0,1646,PRODUCE,National,SALAD MIX,GARDEN PLUS,18 OZ,45-54,...,2,None/Unknown,2417.12,35.916667,1.985714,3.397050,1.891892,2510.789474,-2508.897582,0.000754
105551,1745,948640,0.0,1213,DRUG GM,National,ORAL HYGIENE PRODUCTS,WHITENING SYSTEMS,3 OZ,45-54,...,2,None/Unknown,2417.12,35.916667,1.157895,6.596818,2.500000,2694.557895,-2692.057895,0.000928


In [107]:
df_ranker_train.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,household_size_desc,kid_category_desc,sales_value,average_month_buy_on_user,average_item_buy_on_week,price,count_per_cat_on_user,count_per_cat,average_subtraction_on_cat,average_division_on_cat
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.745763,3.796408,1.333333,611.484211,-610.150877,0.002180
1,2070,1016298,0.0,4074,DELI,National,CHICKEN/POULTRY,CHIX:VALUE ADDED (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.563636,3.101043,1.333333,611.484211,-610.150877,0.002180
2,2070,8015407,0.0,1216,DELI,National,SANDWICHES,SANDWICHES - (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.971831,2.857000,1.333333,611.484211,-610.150877,0.002180
3,2070,1054185,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.631579,3.797258,1.333333,611.484211,-610.150877,0.002180
4,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,...,1,None/Unknown,6981.86,90.416667,1.275000,11.460588,18.939024,15989.789474,-15970.850449,0.001184


In [108]:
df_ranker_train.isna().sum()

user_id                           0
item_id                           0
target                            0
manufacturer                      0
department                        0
brand                             0
commodity_desc                    0
sub_commodity_desc                0
curr_size_of_product              0
age_desc                      66144
marital_status_code           66144
income_desc                   66144
homeowner_desc                66144
hh_comp_desc                  66144
household_size_desc           66144
kid_category_desc             66144
sales_value                       0
average_month_buy_on_user         0
average_item_buy_on_week          0
price                             0
count_per_cat_on_user             0
count_per_cat                     0
average_subtraction_on_cat        0
average_division_on_cat           0
dtype: int64

In [109]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [110]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'sales_value',
 'average_month_buy_on_user',
 'average_item_buy_on_week',
 'price',
 'count_per_cat_on_user',
 'count_per_cat',
 'average_subtraction_on_cat',
 'average_division_on_cat']

In [111]:

#Смотрим Null в датафрейме, перед тем как класть в модель
df_ranker_train.isna().sum()

user_id                           0
item_id                           0
target                            0
manufacturer                      0
department                        0
brand                             0
commodity_desc                    0
sub_commodity_desc                0
curr_size_of_product              0
age_desc                      66144
marital_status_code           66144
income_desc                   66144
homeowner_desc                66144
hh_comp_desc                  66144
household_size_desc           66144
kid_category_desc             66144
sales_value                       0
average_month_buy_on_user         0
average_item_buy_on_week          0
price                             0
count_per_cat_on_user             0
count_per_cat                     0
average_subtraction_on_cat        0
average_division_on_cat           0
dtype: int64

In [112]:
#Чистим null, т.к null у категориальных переменных заменим их Unknown, просто как отдельная категория
df_ranker_train = df_ranker_train.fillna('Unknown')
df_ranker_train.isna().sum()

user_id                       0
item_id                       0
target                        0
manufacturer                  0
department                    0
brand                         0
commodity_desc                0
sub_commodity_desc            0
curr_size_of_product          0
age_desc                      0
marital_status_code           0
income_desc                   0
homeowner_desc                0
hh_comp_desc                  0
household_size_desc           0
kid_category_desc             0
sales_value                   0
average_month_buy_on_user     0
average_item_buy_on_week      0
price                         0
count_per_cat_on_user         0
count_per_cat                 0
average_subtraction_on_cat    0
average_division_on_cat       0
dtype: int64

In [100]:
#Средние недельные покупки товара
item_buy_in_week = data.groupby(['item_id', 'week_no']).size().reset_index()
item_buy_in_week = item_buy_in_week.groupby('item_id')[0].mean().reset_index()
item_buy_in_week.rename(columns={0: 'average_item_buy_on_week'}, inplace=True)
df_ranker_train = df_ranker_train.merge(item_buy_in_week, on='item_id', how='left')
df_ranker_train

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,sales_value,average_month_buy_on_user,average_item_buy_on_week
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667,1.745763
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667,1.275000
2,2070,879194,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667,1.352941
3,2070,948640,0.0,1213,DRUG GM,National,ORAL HYGIENE PRODUCTS,WHITENING SYSTEMS,3 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667,1.157895
4,2070,928263,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,13 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667,1.718750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106967,1745,903454,0.0,1216,MEAT-PCKGD,National,FROZEN MEAT,OTHER - FULLY COOKED,32 OZ,45-54,A,Under 15K,Unknown,Single Male,2,None/Unknown,2417.12,35.916667,2.181818
106968,1745,9419888,0.0,759,GROCERY,National,YOGURT,YOGURT MULTI-PACKS,48 OZ,45-54,A,Under 15K,Unknown,Single Male,2,None/Unknown,2417.12,35.916667,1.524590
106969,1745,1076769,0.0,3859,DELI,National,DELI MEATS,MEAT: LUNCHMEAT BULK,,45-54,A,Under 15K,Unknown,Single Male,2,None/Unknown,2417.12,35.916667,1.946667
106970,1745,1092588,0.0,709,GROCERY,National,FLUID MILK PRODUCTS,MISCELLANEOUS MILK,32 OZ,45-54,A,Under 15K,Unknown,Single Male,2,None/Unknown,2417.12,35.916667,3.617978


In [55]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [56]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

**Обучение модели ранжирования**

In [57]:
# делали на вебинаре, кусок с катбустом

# from catboost import CatBoost, Pool

# df_bin_feat = pd.get_dummies(X_train)

# model = CatBoost(params ={'objective': "Logloss",
#                           "iterations":1000,
#                           "max_depth":8,
#                           "task_type":"GPU",
#                           "devices":"0:1:2:3"
#             })
# model.fit(df_bin_juice, y_train, silent=True)

# train_preds = model.predict(df_bin_feat,prediction_type="Probability")

In [113]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=8,
                     n_estimators=300,
                     learning_rate=0.05,
                     categorical_column=cat_feats,
                     n_jobs=-1,
#                      verbose=0
                   )

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


In [114]:
df_ranker_predict = df_ranker_train.copy()

In [115]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [116]:
df_ranker_predict['proba_item_purchase'][:10]

0    0.009540
1    0.080197
2    0.017918
3    0.004257
4    0.017458
5    0.049668
6    0.064285
7    0.038985
8    0.054960
9    0.056168
Name: proba_item_purchase, dtype: float64

**Подведем итоги**

*Мы обучили модель ранжирования на покупках из сета data_train_ranker и на кандитатах от own_recommendations, что является тренировочным сетом, и теперь наша задача предсказать и оценить именно на тестовом сете.*

**Evaluation on test dataset**

In [117]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


**Eval matching on test dataset¶**

In [118]:
%%time
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 12.5 s, sys: 89.9 ms, total: 12.6 s
Wall time: 12.5 s


In [119]:
# померяем precision только модели матчинга, чтобы понимать влияение ранжирования на метрики

sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

[('own_rec', 0.1444117647058813)]

**Eval re-ranked matched result on test dataset**


*Вспомним df_match_candidates сет, который был получен own_recommendations на юзерах, набор пользователей мы фиксировали и он одинаков, значи и прогноз одинаков, поэтому мы можем использовать этот датафрейм для переранжирования.*

In [120]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [121]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

In [122]:
print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.16783289817232225)
('own_rec', 0.1444117647058813)


/content/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


***ВЫВОД: безусловно, двухуровневая модель дает прирост качества, при этом достаточно ощутимый.***